# Оценка линейного искажающего оператора в задаче восстановления изображений
Время работы: очень долго(больше часа)

In [ ]:
import numpy as np
from skimage import color, data, restoration, img_as_float, io
# import matplotlib.pyplot as plt
from scipy.signal import convolve2d as conv2
from skimage.measure import compare_psnr
from functools import partial
from scipy.optimize import minimize
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from tqdm import tqdm_notebook
import pickle
import os.path

from util import *
from richardson_lucy import *
plt.gray()

In [ ]:
prefix="pic/cgrid"

In [ ]:
astro = color.rgb2gray(data.astronaut())
liftingbody = img_as_float(io.imread('liftingbody.png'))

In [ ]:
iterations=20

## Оценка криволинейного оператора смаза, представленного кривой Безье

In [ ]:
sz = 21
deg = 5
points_real = [(deg-1)*sz//deg-1, sz//deg, sz-1, sz-1]
print(points_real)
psf_bezier = bezier_psf2(points_real, 100)
plt.imshow(psf_bezier)
plt.title(np.concatenate(([0,0], points_real)))
plt.plot(0, 0, 'ro')
plt.plot(points_real[0], points_real[1], 'ro')
plt.plot(points_real[2], points_real[3], 'ro')
plt.savefig("{0}-curved-psf.png".format(prefix))
plt.show()

In [ ]:
lifting_bezier_blurred = convolve2d(liftingbody, psf_bezier, 'same')
#lifting_bezier_blurred = edgetaper(lifting_bezier_blurred, gkern2(41,10))

In [ ]:
lifting_restored = restoration.richardson_lucy(lifting_bezier_blurred, psf_bezier, iterations=iterations)
psnr_py = compare_psnr_crop(liftingbody, lifting_restored)

deconv_bezier = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, clip=True, useFFT=True, dampar=0.004)
psnr_my = compare_psnr_crop(liftingbody, deconv_bezier['image'])

In [ ]:
show_results(lifting_bezier_blurred, lifting_restored, deconv_bezier['image'],
            titles=['Blurred image', 'Restored with python LR\nPSNR={0}'.format(psnr_py), 'Restored with my LR\nPSNR={0}'.format(psnr_my)])

### Кепстр

In [ ]:
s_n, S_find = find_noise(lifting_bezier_blurred)
print(s_n, S_find)

In [ ]:
#N,M = lifting_bezier_blurred.shape
# ( ifft2 (100* log (1+ abs ( fft2 ( I ) ) ) ) ) ;
K = np.real(np.fft.ifftn(100*np.log(1+np.abs(np.fft.fftn(lifting_bezier_blurred)))))
#K_shift = K
K_shift = np.fft.fftshift(K)

In [ ]:
r,c = np.unravel_index(np.argmin(K_shift, axis=None), K_shift.shape)
nn = np.argmin(np.real(K_shift))
n, m, = K_shift.shape
x2 = np.array([c - n//2, r - m//2])
if x2[0]<0:
    x2 = -x2
x1 = x2/2
print(x2)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.clip(np.real(K_shift),-1,1))#, vmin=noisy.min(), vmax=noisy.max())
plt.plot(n//2 + x2[0], m//2 + x2[1], 'ro')
plt.title('Кепстр изображения с выделенным минмиумом')
plt.savefig('{0}-kepstr.png'.format(prefix))
plt.show()

In [ ]:
x0 = np.concatenate((x1, x2))
print(x0)

### Минимизация

In [ ]:
def funcToMinimizeCurved(xy, I_blurred, crop=100, *args, **kwargs):
    print(xy)
    psf = bezier_psf2(xy, n=100)
    restored = richardson_lucy_matlab(I_blurred, psf, *args, **kwargs)
    I_restored = restored['image']
    df = fftconvolve(I_restored, psf, 'same') - I_blurred
    return np.mean(np.square(df[crop:-crop, crop:-crop]))
#partial(funcToMinimizeCurved, I_blurred=lifting_bezier_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)([0, 10, 10, 10])

In [ ]:
def funcToMinimizeCurved2(xy, I_blurred, xy2, crop=100, *args, **kwargs):
    psf = bezier_psf2(np.concatenate((np.array(xy), np.array(xy2))))
    restored = richardson_lucy_matlab(I_blurred, psf, *args, **kwargs)
    I_restored = restored['image']
    df = convolve2d(I_restored, psf, 'same') - I_blurred
    return np.mean(np.square(df[crop:-crop, crop:-crop]))

In [ ]:
f = partial(funcToMinimizeCurved2, I_blurred=lifting_bezier_blurred, xy2=x2, iterations=iterations, eps=1e-5, dampar=10*s_n, clip=True, useFFT=True)

Scale=2 => 1 час <img src='pic/optimization_useless_x2.png'>
Scale=4 => 5 часов <img src='pic/optimization_useless_x4.png'>

In [ ]:
pickle_filename = "matrix4.pickle"
scale = 4
if os.path.isfile(pickle_filename):
    with open(pickle_filename, 'rb') as a_file:
        a = pickle.load(a_file)
else:
    a = np.ones((scale*x2[1], scale*x2[0]))
    for yy in tqdm_notebook(range(scale*x2[1])):
        for xx in range(scale*x2[0]):
            val = f(np.array([xx/scale,yy/scale]))
            print(yy/scale, xx/scale, val)
            a[yy,xx] = val
    with open(pickle_filename, 'wb') as a_file:
        pickle.dump(a, a_file)

In [ ]:
r, c = np.unravel_index(np.argmin(a, axis=None), a.shape)

In [ ]:
plt.figure(figsize=(6,6))
fig = plt.imshow(a, 'gray')
plt.title('Cost function value for second point coordinates,\ndarker is better')
plt.plot(points_real[0]*scale, points_real[1]*scale, 'ro')
plt.plot(c, r,'go')
plt.legend(['real','min'], loc='lower right')
fig.axes.get_xaxis().set_visible(False)
fig.axes.get_yaxis().set_visible(False)
plt.savefig('{0}-optimization_useless_x{1}.png'.format(prefix, scale))
plt.show()

In [ ]:
points_found = np.concatenate(([c/scale,r/scale], x2))#[(deg-1)*sz//deg-1, sz//deg, sz-1, sz-1]
print(points_found)
psf_found = bezier_psf2(points_found)

In [ ]:
f([c/scale,r/scale]), f(points_real[:2])

In [ ]:
#%matplotlib notebook
%matplotlib inline

from mpl_toolkits.mplot3d import Axes3D
X, Y = np.meshgrid(range(a.shape[1]),range(a.shape[0]))

fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, a)
plt.show()

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, dampar=s_n, useFFT=False)
psnr_orig = compare_psnr_crop(liftingbody, deconv_orig['image'])

psf_init = bezier_psf2(x0)
deconv_init = richardson_lucy_matlab(lifting_bezier_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr_crop(liftingbody, deconv_init['image'])

#points_found = res_tnc['x']
#psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr_crop(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig.round(2), points_real), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init.round(2), x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found.round(2), points_found)],
            savefig="{0}-curved-grid.png".format(prefix))

In [ ]:
#x0 = [12, 10, 20, 20]
res_nelder = minimize(partial(funcToMinimizeCurved2, I_blurred=lifting_bezier_blurred, xy2=x2, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
                      x0=x1, method='Nelder-Mead', options={'xtol': 1e-3, 'disp': True})
print(res_nelder)

In [ ]:
x1, points_real

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, dampar=s_n, useFFT=False)
psnr_orig = compare_psnr_crop(liftingbody, deconv_orig['image'])

psf_init = bezier_psf2(x0)
deconv_init = richardson_lucy_matlab(lifting_bezier_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr_crop(liftingbody, deconv_init['image'])

points_found = res_nelder['x']
psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr_crop(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig.round(2), points_real), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init.round(2), x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found.round(2), res_nelder['x'])])

* Запустить градиентный метод
* Сделать мощный edgetaper
* Сделать антиалиасинг psf